1. Read the data from “assign_chubb_engineering .json” and convert it into a table with columns:

In [ ]:
!pip install gdown --upgrade --quiet # Fixed file access permission problem.

Importing json data and python libraries

In [ ]:
import pandas as pd
import json
import gdown

In [ ]:
url = "https://drive.google.com/file/d/14WFGedwSKNqX4WGwnnN4tzB6cvQZtKpC/view?usp=share_link"
output = 'company_data.json'
gdown.download(url, output, fuzzy=True )

Downloading...
From: https://drive.google.com/uc?id=14WFGedwSKNqX4WGwnnN4tzB6cvQZtKpC
To: /content/company_data.json
100%|██████████| 288M/288M [00:02<00:00, 98.8MB/s]


'company_data.json'

In [ ]:
data = json.load(open("/content/company_data.json", "r"))

In [ ]:
df = pd.DataFrame.from_dict(data, orient='index')

Working on json data to get desired data in form of pandas dataframe

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.rename({'index': 'company_names'}, axis=1, inplace=True)

In [ ]:
com_hired = df.companies_they_hire_from.apply(pd.Series)

In [ ]:
df = df.join(com_hired['count'].rename('employee_count_they_hired').astype('int32'), how='left')

In [ ]:
df.drop('companies_they_hire_from', axis=1, inplace=True)

Exploding 'company_name' into 3 parts to limit high RAM usage and taking union

In [ ]:
temp1 = com_hired.company_name.iloc[:5466].apply(pd.Series).stack().to_frame()

In [ ]:
one = temp1[0].apply(pd.Series).rename(columns={'count':'employee_count_they_hired'}).employee_count_they_hired.astype('int32')

In [ ]:
temp2 = com_hired.company_name.iloc[5466:27330].apply(pd.Series).stack().to_frame()

In [ ]:
two = temp2[0].apply(pd.Series).rename(columns={'count':'employee_count_they_hired'}).employee_count_they_hired.astype('int32')

In [ ]:
temp3 = com_hired.company_name.iloc[27330:].apply(pd.Series).stack().to_frame()

In [ ]:
three = temp3[0].apply(pd.Series).rename(columns={'count':'employee_count_they_hired'}).employee_count_they_hired.astype('int32')

In [ ]:
com_hired_final = pd.concat([one, two, three]).to_frame()

In [ ]:
com_hired_final = com_hired_final.reset_index(level=1).rename({'level_1':'list_of_companies_they_hired_from'}, axis=1)

In [ ]:
college = df.employees.apply(pd.Series)

In [ ]:
df = df.join(college[['count', 'freshers_count']].astype('int32').rename(columns={'count':'employees_count'}), how='left')

In [ ]:
df.drop('employees', axis=1, inplace=True)

In [ ]:
college.drop(['count', 'freshers_count', 'freshers_uuid', 'employees_uuid'], axis=1, inplace=True)

In [ ]:
iit_college = college['from_iit'].apply(pd.Series).rename(columns={'count':'iit_count', 'freshers_count':'iit_freshers_count'})[['iit_count','iit_freshers_count']].astype('int32')

In [ ]:
bits_college = college['from_bits'].apply(pd.Series).rename(columns={'count':'bits_count', 'freshers_count':'bits_freshers_count'})[['bits_count','bits_freshers_count']].astype('int32')

In [ ]:
nit_college = college['from_nit'].apply(pd.Series).rename(columns={'count':'nit_count', 'freshers_count':'nit_freshers_count'})[['nit_count','nit_freshers_count']].astype('int32')

In [ ]:
df = df.join([iit_college, bits_college, nit_college], how='left')

In [ ]:
ex_employees = df['companies_where_their_ex_employees_work'].apply(pd.Series)

In [ ]:
ex_employees = ex_employees.astype({'count':'int32'}).rename({'count':'employee_count_their_ex-employees_work'}, axis=1)

In [ ]:
temp1 = ex_employees.company_name.iloc[:10036].apply(pd.Series).stack().to_frame()

In [ ]:
one = temp1[0].apply(pd.Series).rename(columns={'count':'employee_count_their_ex-employees_work'})['employee_count_their_ex-employees_work'].astype('int32')

In [ ]:
temp2 = ex_employees.company_name.iloc[10036:25090].apply(pd.Series).stack().to_frame()

In [ ]:
two = temp2[0].apply(pd.Series).rename(columns={'count':'employee_count_their_ex-employees_work'})['employee_count_their_ex-employees_work'].astype('int32')

In [ ]:
temp3 = ex_employees.company_name.iloc[25090:].apply(pd.Series).stack().to_frame()

In [ ]:
three = temp3[0].apply(pd.Series).rename(columns={'count':'employee_count_their_ex-employees_work'})['employee_count_their_ex-employees_work'].astype('int32')

In [ ]:
ex_employees_final = pd.concat([one, two, three]).to_frame()

In [ ]:
ex_employees_final = ex_employees_final.reset_index(level=1).rename({'level_1':'list_of_companies_their_ex-employees_work'}, axis=1)

In [ ]:
df.drop('companies_where_their_ex_employees_work', axis=1, inplace=True)

In [ ]:
df.rename(columns={'Department_Tier-1_College_Density_Score': 'department_tier-1_college_density_score', 'Department_Tier-1_College_Freshers_Density_Score': 'department_tier-1_college_freshers_density_score'}, inplace=True)

### Converted json data into 3 different tables(dataframes)

*   df - Contains company names, employees count, employees hired count, all college data
*   com_hired_final - Contains list_of_companies_they_hired_from, 	employee_count_they_hired
*   ex_employees_final - list_of_companies_their_ex-employees_work, 	employee_count_their_ex-employees_work



In [ ]:
df

,company_names,company_uuid,is_score_calculated,department_tier-1_college_density_score,department_tier-1_college_freshers_density_score,employee_count_they_hired,employees_count,freshers_count,iit_count,iit_freshers_count,bits_count,bits_freshers_count,nit_count,nit_freshers_count
0,altran technologies india pvt ltd,None,False,0.000010,0.000010,1,1,0,0,0,0,0,0,0
1,L&T Technology Services Ltd.,,True,0.004499,0.004640,4019,6890,2155,16,7,2,0,13,3
2,Amdocs,fbd567c5e7395b46974ece634a19ff2e,False,0.005260,0.035714,1614,1711,28,4,0,0,0,5,1
3,Xperia,7aa9a32dd8f27a05646e7317af05959b,False,0.000010,0.000010,6,6,0,0,0,0,0,0,0
4,Ultra-Scan Corporation,,False,0.071930,0.000010,1102,1140,8,63,0,2,0,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60121,babu it jobs,None,False,0.000010,0.000010,1,1,0,0,0,0,0,0,0
60122,capital honda service centre,,False,0.000010,0.000010,0,0,0,0,0,0,0,0,0
60123,systems plus llc,,False,0.000010,0.000010,0,0,0,0,0,0,0,0,0
60124,gemini traze rfid pvt ltd,,False,0.000010,0.000010,0,0,0,0,0,0,0,0,0


In [ ]:
com_hired_final

,list_of_companies_they_hired_from,employee_count_they_hired
0,L&T Technology Services Ltd.,1
1,HCL Technologies,86
1,Tech Mahindra,85
1,Wipro Technologies,80
1,Tata Consultancy Services,80
...,...,...
60107,Tech Mahindra,1
60108,Tech Mahindra,1
60112,DHI,1
60118,Tech Mahindra,1


In [ ]:
ex_employees_final

,list_of_companies_their_ex-employees_work,employee_count_their_ex-employees_work
1,Robert Bosch Engineering & Business Solutions,63
1,Harman International Industries,63
1,Synamedia,60
1,Ultra-Scan Corporation,52
1,Wipro Technologies,35
...,...,...
60120,Tech Mahindra,1
60122,Tech Mahindra,1
60123,Tech Mahindra,1
60124,Tech Mahindra,1


In [ ]:
#df.to_parquet("df_main.parquet")

In [ ]:
#com_hired_final.to_parquet('com_hired_final.parquet')

In [ ]:
#ex_employees_final.to_parquet('ex_employees_final.parquet')

In [ ]:
# import pandas as pd
# import json
# import gdown

In [ ]:
# df = pd.read_parquet('df_main.parquet')
# com_hired_final = pd.read_parquet('com_hired_final.parquet')
# ex_employees_final = pd.read_parquet('ex_employees.parquet')

2 - Find number of null values for each column

In [ ]:
combo_df = df[['company_names']].join([ex_employees_final, com_hired_final], how='left')

In [ ]:
combo_df.isna().sum()

company_names                                    0
list_of_companies_their_ex-employees_work    20507
employee_count_their_ex-employees_work       20507
list_of_companies_they_hired_from            42479
employee_count_they_hired                    42479
dtype: int64

In [ ]:
df.isna().sum()

company_names                                           0
company_uuid                                        12449
is_score_calculated                                     0
department_tier-1_college_density_score                 0
department_tier-1_college_freshers_density_score        0
employee_count_they_hired                               0
employees_count                                         0
freshers_count                                          0
iit_count                                               0
iit_freshers_count                                      0
bits_count                                              0
bits_freshers_count                                     0
nit_count                                               0
nit_freshers_count                                      0
dtype: int64

In [ ]:
com_hired_final.isna().sum()

list_of_companies_they_hired_from    0
employee_count_they_hired            0
dtype: int64

In [ ]:
ex_employees_final.isna().sum()

list_of_companies_their_ex-employees_work    0
employee_count_their_ex-employees_work       0
dtype: int64

3. Add a column ‘List of companies where hiring companies hire from’. It contains companies where
the hiring companies of current company hires its employees. For example, company A hire from
companies (B, C, D) then ‘List of companies where hiring companies hire from’ for company A has all
the companies where B or C or D companies hire

In [ ]:
df[['company_names']].join(com_hired_final[['list_of_companies_they_hired_from']], how='left')

,company_names,list_of_companies_they_hired_from
0,altran technologies india pvt ltd,L&T Technology Services Ltd.
1,L&T Technology Services Ltd.,HCL Technologies
1,L&T Technology Services Ltd.,Tech Mahindra
1,L&T Technology Services Ltd.,Wipro Technologies
1,L&T Technology Services Ltd.,Tata Consultancy Services
...,...,...
60121,babu it jobs,Tech Mahindra
60122,capital honda service centre,NaN
60123,systems plus llc,NaN
60124,gemini traze rfid pvt ltd,NaN


4. Based on Number of employees classify them into 9 categories and add a column with category
number

In [ ]:
def categorise(cnt):
  if cnt in range(1, 11):
    return 1
  elif cnt in range(11, 51):
    return 2
  elif cnt in range(51, 101):
    return 3
  elif cnt in range(101, 251):
    return 4
  elif cnt in range(251, 501):
    return 5
  elif cnt in range(501, 1001):
    return 6
  elif cnt in range(1001, 5001):
    return 7
  elif cnt in range(5001, 10001):
    return 8
  elif cnt > 10000:
    return 9

In [ ]:
df['category_number'] = df['employees_count'].apply(categorise)

5. Find number of companies for each category

In [ ]:
df[['category_number', 'company_names']].groupby(by=['category_number']).count()

,company_names
category_number,
1.0,24362
2.0,1427
3.0,280
4.0,216
5.0,69
6.0,38
7.0,30
8.0,4
9.0,7


6. Find median and mean of tier-1 strength for each category

In [ ]:
df[['category_number', 'department_tier-1_college_density_score']].groupby(by=['category_number']).median()

,department_tier-1_college_density_score
category_number,
1.0,0.000010
2.0,0.000010
3.0,0.000010
4.0,0.008163
5.0,0.011070
6.0,0.008969
7.0,0.010643
8.0,0.005113
9.0,0.003593


In [ ]:
df[['category_number', 'department_tier-1_college_density_score']].groupby(by=['category_number']).mean()

,department_tier-1_college_density_score
category_number,
1.0,0.019664
2.0,0.016661
3.0,0.016730
4.0,0.013943
5.0,0.016181
6.0,0.012446
7.0,0.018061
8.0,0.010760
9.0,0.003900


7. Display top 5 and bottom 5 companies based on Tier-1 College Freshers Strength for each category

In [ ]:
df[['company_names', 'category_number', 'department_tier-1_college_density_score']].sort_values(by=['category_number', 'department_tier-1_college_density_score'], ascending=[True, False]).groupby('category_number', sort=False).head(5)

,company_names,category_number,department_tier-1_college_density_score
13990,Climate Connect Technologies,1.0,2.000000
1009,APSIS Solutions,1.0,1.000000
1173,Central Electronics Limited,1.0,1.000000
2309,ambient scientific,1.0,1.000000
3104,Activation Energy,1.0,1.000000
20535,ShareChat,2.0,0.333333
16288,Harness,2.0,0.285714
16779,Deserve,2.0,0.272727
4747,Government of India,2.0,0.266667
18772,Socure,2.0,0.250000


In [ ]:
df[['company_names', 'category_number', 'department_tier-1_college_density_score']].sort_values(by=['category_number', 'department_tier-1_college_density_score'], ascending=[True, False]).groupby('category_number', sort=False).tail(5)

,company_names,category_number,department_tier-1_college_density_score
60107,philips india,1.0,0.000010
60108,Evolution Digital,1.0,0.000010
60112,soczen technology,1.0,0.000010
60118,apl mechanical services limited,1.0,0.000010
60121,babu it jobs,1.0,0.000010
27889,Perfios Software Solutions,2.0,0.000010
28653,Fujitsu Network Communications,2.0,0.000010
32846,Vidal Healthcare,2.0,0.000010
33878,Glu Mobile,2.0,0.000010
45495,pandora r&d labs private limited,2.0,0.000010


In [ ]:
df

,company_names,company_uuid,is_score_calculated,department_tier-1_college_density_score,department_tier-1_college_freshers_density_score,employee_count_they_hired,employees_count,freshers_count,iit_count,iit_freshers_count,bits_count,bits_freshers_count,nit_count,nit_freshers_count,category_number
0,altran technologies india pvt ltd,None,False,0.000010,0.000010,1,1,0,0,0,0,0,0,0,1.0
1,L&T Technology Services Ltd.,,True,0.004499,0.004640,4019,6890,2155,16,7,2,0,13,3,8.0
2,Amdocs,fbd567c5e7395b46974ece634a19ff2e,False,0.005260,0.035714,1614,1711,28,4,0,0,0,5,1,7.0
3,Xperia,7aa9a32dd8f27a05646e7317af05959b,False,0.000010,0.000010,6,6,0,0,0,0,0,0,0,1.0
4,Ultra-Scan Corporation,,False,0.071930,0.000010,1102,1140,8,63,0,2,0,17,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60121,babu it jobs,None,False,0.000010,0.000010,1,1,0,0,0,0,0,0,0,1.0
60122,capital honda service centre,,False,0.000010,0.000010,0,0,0,0,0,0,0,0,0,NaN
60123,systems plus llc,,False,0.000010,0.000010,0,0,0,0,0,0,0,0,0,NaN
60124,gemini traze rfid pvt ltd,,False,0.000010,0.000010,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
df[df['category_number'] == 1].join([com_hired_final, ex_employees_final], how='left')

,company_names,company_uuid,is_score_calculated,department_tier-1_college_density_score,department_tier-1_college_freshers_density_score,employee_count_they_hired_x,employees_count,freshers_count,iit_count,iit_freshers_count,bits_count,bits_freshers_count,nit_count,nit_freshers_count,category_number,list_of_companies_they_hired_from,employee_count_they_hired_y,list_of_companies_their_ex-employees_work,employee_count_their_ex-employees_work
0,altran technologies india pvt ltd,None,False,0.00001,0.00001,1,1,0,0,0,0,0,0,0,1.0,L&T Technology Services Ltd.,1.0,NaN,NaN
3,Xperia,7aa9a32dd8f27a05646e7317af05959b,False,0.00001,0.00001,6,6,0,0,0,0,0,0,0,1.0,L&T Technology Services Ltd.,1.0,Cisco,1.0
3,Xperia,7aa9a32dd8f27a05646e7317af05959b,False,0.00001,0.00001,6,6,0,0,0,0,0,0,0,1.0,Ultra-Scan Corporation,3.0,Cisco,1.0
3,Xperia,7aa9a32dd8f27a05646e7317af05959b,False,0.00001,0.00001,6,6,0,0,0,0,0,0,0,1.0,Telescope Services AB,2.0,Cisco,1.0
5,Adani Gas,,False,0.00001,0.00001,4,4,0,0,0,0,0,0,0,1.0,Tata Consultancy Services,1.0,Wipro Technologies,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60107,philips india,None,False,0.00001,0.00001,1,1,0,0,0,0,0,0,0,1.0,Tech Mahindra,1.0,NaN,NaN
60108,Evolution Digital,None,False,0.00001,0.00001,1,1,0,0,0,0,0,0,0,1.0,Tech Mahindra,1.0,NaN,NaN
60112,soczen technology,None,False,0.00001,0.00001,1,1,0,0,0,0,0,0,0,1.0,DHI,1.0,NaN,NaN
60118,apl mechanical services limited,None,False,0.00001,0.00001,1,1,0,0,0,0,0,0,0,1.0,Tech Mahindra,1.0,NaN,NaN


8. Save the data category wise into separate excel sheet and name it as category_k.xlsx where k is the
category number

In [ ]:
for x in range(1,10):
  df_excel = df[df['category_number'] == x].join([com_hired_final, ex_employees_final], how='left')
  filename = f'category_{x}.xlsx'
  df_excel.to_excel(filename, header=True, index=False)